In [246]:
import pandas as pd

In [247]:
ride_data = pd.read_csv('./data/ncr_ride_bookings.csv')
weather_data = pd.read_csv('./data/weather_data.csv')
ride_data.head()

,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI


In [248]:
ride_data['event_dt'] = pd.to_datetime(
    ride_data['Date'].astype(str).str.strip() + ' ' + ride_data['Time'].astype(str).str.strip(),
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)
ride_data['event_time'] = ride_data['event_dt'].dt.time

In [249]:
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'], errors='coerce')

ride_sorted = ride_data.sort_values('event_dt').reset_index(drop=True)
weather_sorted = weather_data.sort_values('datetime').reset_index(drop=True)

# 3) Nearest-time merge (set tolerance as needed, e.g., 30min/60min)
tolerance = pd.Timedelta('30min')  # max allowed gap between ride and weather time
rides_with_weather = pd.merge_asof(
    ride_sorted,
    weather_sorted,
    left_on='event_dt',
    right_on='datetime',
    direction='nearest',
    tolerance=tolerance,
    suffixes=('', '_wx')
)

rides_with_weather

,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations,raining
0,2024-01-01,00:19:34,"""CNR4352144""",Completed,"""CID8362794""",Bike,Udyog Vihar,Ambience Mall,10.8,38.9,...,0.0,0.6,0.0,0.0,0,NaN,Clear,fog,"VIDP,42181099999",False
1,2024-01-01,01:35:18,"""CNR9147645""",Completed,"""CID8300238""",Go Mini,Basai Dhankot,Madipur,8.5,15.1,...,1.2,0.6,0.0,0.0,0,NaN,Clear,fog,"42182099999,VIDP,42181099999",False
2,2024-01-01,01:37:50,"""CNR1009222""",Cancelled by Driver,"""CID2030746""",Go Sedan,Tughlakabad,Greater Kailash,7.4,NaN,...,1.2,0.6,0.0,0.0,0,NaN,Clear,fog,"42182099999,VIDP,42181099999",False
3,2024-01-01,01:48:03,"""CNR2740479""",Cancelled by Driver,"""CID3231181""",Auto,Palam Vihar,Kherki Daula Toll,5.6,NaN,...,1.2,0.6,0.0,0.0,0,NaN,Clear,fog,"42182099999,VIDP,42181099999",False
4,2024-01-01,01:49:56,"""CNR7650148""",Cancelled by Driver,"""CID3381661""",Go Sedan,Narsinghpur,Pulbangash,6.2,NaN,...,1.2,0.6,0.0,0.0,0,NaN,Clear,fog,"42182099999,VIDP,42181099999",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,2024-12-30,22:58:00,"""CNR9030698""",Completed,"""CID6939658""",Bike,DLF Phase 3,Okhla,6.9,35.8,...,29.2,0.6,0.0,0.0,0,NaN,Partially cloudy,fog,"42182099999,VIDP,42181099999",False
149996,2024-12-30,23:03:14,"""CNR1507833""",Completed,"""CID9905090""",eBike,Saket,Noida Sector 62,7.9,33.6,...,29.2,0.6,0.0,0.0,0,NaN,Partially cloudy,fog,"42182099999,VIDP,42181099999",False
149997,2024-12-30,23:17:05,"""CNR2265113""",Completed,"""CID4170406""",Go Mini,GTB Nagar,Anand Vihar ISBT,9.9,39.9,...,29.2,0.6,0.0,0.0,0,NaN,Partially cloudy,fog,"42182099999,VIDP,42181099999",False
149998,2024-12-30,23:21:12,"""CNR4066733""",Completed,"""CID8938559""",Uber XL,Ashram,Vasant Kunj,2.6,26.7,...,29.2,0.6,0.0,0.0,0,NaN,Partially cloudy,fog,"42182099999,VIDP,42181099999",False


In [250]:
rides_with_weather = rides_with_weather.drop(columns=['event_dt', 'datetime', 'event_time'])

In [251]:
rides_with_weather['datetime'] = pd.to_datetime(rides_with_weather['Date'] + ' ' + rides_with_weather['Time'])
rides_with_weather['hour'] = rides_with_weather['datetime'].dt.hour
rides_with_weather['minute'] = rides_with_weather['datetime'].dt.minute

In [252]:
rides_with_weather.drop_duplicates(subset=["Booking ID"], inplace=True)

In [253]:
df = rides_with_weather.drop(columns=[
    'Date', 'Time', 'datetime', 'Booking ID', 'Customer ID', 'Cancelled Rides by Customer', 'Cancelled Rides by Driver', 'Incomplete Rides',
    'stations', 'icon', 'snow', 'snowdepth', 'precipprob', 'precip', 'Payment Method'
])

In [254]:
df

,Booking Status,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,Reason for cancelling by Customer,Driver Cancellation Reason,Incomplete Rides Reason,Booking Value,...,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,raining,hour,minute
0,Completed,Bike,Udyog Vihar,Ambience Mall,10.8,38.9,NaN,NaN,NaN,99.0,...,0.0,0.6,0.0,0.0,0,NaN,Clear,False,0,19
1,Completed,Go Mini,Basai Dhankot,Madipur,8.5,15.1,NaN,NaN,NaN,114.0,...,1.2,0.6,0.0,0.0,0,NaN,Clear,False,1,35
2,Cancelled by Driver,Go Sedan,Tughlakabad,Greater Kailash,7.4,NaN,NaN,More than permitted people in there,NaN,NaN,...,1.2,0.6,0.0,0.0,0,NaN,Clear,False,1,37
3,Cancelled by Driver,Auto,Palam Vihar,Kherki Daula Toll,5.6,NaN,NaN,Personal & Car related issues,NaN,NaN,...,1.2,0.6,0.0,0.0,0,NaN,Clear,False,1,48
4,Cancelled by Driver,Go Sedan,Narsinghpur,Pulbangash,6.2,NaN,NaN,More than permitted people in there,NaN,NaN,...,1.2,0.6,0.0,0.0,0,NaN,Clear,False,1,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,Completed,Bike,DLF Phase 3,Okhla,6.9,35.8,NaN,NaN,NaN,440.0,...,29.2,0.6,0.0,0.0,0,NaN,Partially cloudy,False,22,58
149996,Completed,eBike,Saket,Noida Sector 62,7.9,33.6,NaN,NaN,NaN,279.0,...,29.2,0.6,0.0,0.0,0,NaN,Partially cloudy,False,23,3
149997,Completed,Go Mini,GTB Nagar,Anand Vihar ISBT,9.9,39.9,NaN,NaN,NaN,1855.0,...,29.2,0.6,0.0,0.0,0,NaN,Partially cloudy,False,23,17
149998,Completed,Uber XL,Ashram,Vasant Kunj,2.6,26.7,NaN,NaN,NaN,520.0,...,29.2,0.6,0.0,0.0,0,NaN,Partially cloudy,False,23,21


In [255]:
from sklearn.preprocessing import OrdinalEncoder

cat_cols = ['Vehicle Type', 'Pickup Location', 'Drop Location']
cat_cols = [c for c in cat_cols if c in df.columns]

# Normalize types and handle missing
df[cat_cols] = df[cat_cols].astype('string').fillna('Unknown')

# Encode; unknown categories at transform time -> -1
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df[cat_cols] = encoder.fit_transform(df[cat_cols])

# Optional: keep category lists for later use
encoded_categories = {col: list(cats) for col, cats in zip(cat_cols, encoder.categories_)}

In [256]:
df

,Booking Status,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,Reason for cancelling by Customer,Driver Cancellation Reason,Incomplete Rides Reason,Booking Value,...,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,raining,hour,minute
0,Completed,1.0,165.0,3.0,10.8,38.9,NaN,NaN,NaN,99.0,...,0.0,0.6,0.0,0.0,0,NaN,Clear,False,0,19
1,Completed,2.0,17.0,88.0,8.5,15.1,NaN,NaN,NaN,114.0,...,1.2,0.6,0.0,0.0,0,NaN,Clear,False,1,35
2,Cancelled by Driver,3.0,163.0,42.0,7.4,NaN,NaN,More than permitted people in there,NaN,NaN,...,1.2,0.6,0.0,0.0,0,NaN,Clear,False,1,37
3,Cancelled by Driver,0.0,116.0,81.0,5.6,NaN,NaN,Personal & Car related issues,NaN,NaN,...,1.2,0.6,0.0,0.0,0,NaN,Clear,False,1,48
4,Cancelled by Driver,3.0,102.0,126.0,6.2,NaN,NaN,More than permitted people in there,NaN,NaN,...,1.2,0.6,0.0,0.0,0,NaN,Clear,False,1,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,Completed,1.0,30.0,113.0,6.9,35.8,NaN,NaN,NaN,440.0,...,29.2,0.6,0.0,0.0,0,NaN,Partially cloudy,False,22,58
149996,Completed,6.0,141.0,112.0,7.9,33.6,NaN,NaN,NaN,279.0,...,29.2,0.6,0.0,0.0,0,NaN,Partially cloudy,False,23,3
149997,Completed,2.0,36.0,5.0,9.9,39.9,NaN,NaN,NaN,1855.0,...,29.2,0.6,0.0,0.0,0,NaN,Partially cloudy,False,23,17
149998,Completed,5.0,10.0,169.0,2.6,26.7,NaN,NaN,NaN,520.0,...,29.2,0.6,0.0,0.0,0,NaN,Partially cloudy,False,23,21


In [257]:
df = pd.get_dummies(df, columns=['Booking Status'], prefix='Booking Status')
df

,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,Reason for cancelling by Customer,Driver Cancellation Reason,Incomplete Rides Reason,Booking Value,Ride Distance,...,severerisk,conditions,raining,hour,minute,Booking Status_Cancelled by Customer,Booking Status_Cancelled by Driver,Booking Status_Completed,Booking Status_Incomplete,Booking Status_No Driver Found
0,1.0,165.0,3.0,10.8,38.9,NaN,NaN,NaN,99.0,37.98,...,NaN,Clear,False,0,19,False,False,True,False,False
1,2.0,17.0,88.0,8.5,15.1,NaN,NaN,NaN,114.0,39.29,...,NaN,Clear,False,1,35,False,False,True,False,False
2,3.0,163.0,42.0,7.4,NaN,NaN,More than permitted people in there,NaN,NaN,NaN,...,NaN,Clear,False,1,37,False,True,False,False,False
3,0.0,116.0,81.0,5.6,NaN,NaN,Personal & Car related issues,NaN,NaN,NaN,...,NaN,Clear,False,1,48,False,True,False,False,False
4,3.0,102.0,126.0,6.2,NaN,NaN,More than permitted people in there,NaN,NaN,NaN,...,NaN,Clear,False,1,49,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,1.0,30.0,113.0,6.9,35.8,NaN,NaN,NaN,440.0,12.85,...,NaN,Partially cloudy,False,22,58,False,False,True,False,False
149996,6.0,141.0,112.0,7.9,33.6,NaN,NaN,NaN,279.0,39.36,...,NaN,Partially cloudy,False,23,3,False,False,True,False,False
149997,2.0,36.0,5.0,9.9,39.9,NaN,NaN,NaN,1855.0,38.91,...,NaN,Partially cloudy,False,23,17,False,False,True,False,False
149998,5.0,10.0,169.0,2.6,26.7,NaN,NaN,NaN,520.0,36.37,...,NaN,Partially cloudy,False,23,21,False,False,True,False,False


In [258]:
# #knn imputer for missing values this is very slow
#
# from sklearn.impute import KNNImputer
# import numpy as np
# import pandas as pd
#
# targets = ['Avg VTAT', 'Avg CTAT']
# for c in targets:
#     df[c] = pd.to_numeric(df[c], errors='coerce')
#
# # Use all numeric columns to compute neighbor distances
# num_cols = df.select_dtypes(include=[np.number]).columns
#
# imputer = KNNImputer(
#     n_neighbors=5,          # tweak as needed
#     weights='distance',     # distance-weighted average
#     metric='nan_euclidean'  # handles NaNs during distance calc
# )
#
# imputed = imputer.fit_transform(df[num_cols])
# imputed_df = pd.DataFrame(imputed, columns=num_cols, index=df.index)
#
# df[targets] = imputed_df[targets]


In [259]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
import numpy as np

targets = ['Avg VTAT', 'Avg CTAT']
for c in targets:
    df[c] = pd.to_numeric(df[c], errors='coerce')

num_cols = df.select_dtypes(include=[np.number]).columns

imputer = IterativeImputer(
    estimator=BayesianRidge(),
    max_iter=100,
    initial_strategy='median',
    imputation_order='ascending',
    skip_complete=True,
    sample_posterior=False,
    tol=1e-3,
    random_state=42
)

imputed = imputer.fit_transform(df[num_cols])
imputed_df = pd.DataFrame(imputed, columns=num_cols, index=df.index)

df[targets] = imputed_df[targets]

In [260]:
df

,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,Reason for cancelling by Customer,Driver Cancellation Reason,Incomplete Rides Reason,Booking Value,Ride Distance,...,severerisk,conditions,raining,hour,minute,Booking Status_Cancelled by Customer,Booking Status_Cancelled by Driver,Booking Status_Completed,Booking Status_Incomplete,Booking Status_No Driver Found
0,1.0,165.0,3.0,10.8,38.900000,NaN,NaN,NaN,99.0,37.98,...,NaN,Clear,False,0,19,False,False,True,False,False
1,2.0,17.0,88.0,8.5,15.100000,NaN,NaN,NaN,114.0,39.29,...,NaN,Clear,False,1,35,False,False,True,False,False
2,3.0,163.0,42.0,7.4,29.286100,NaN,More than permitted people in there,NaN,NaN,NaN,...,NaN,Clear,False,1,37,False,True,False,False,False
3,0.0,116.0,81.0,5.6,29.016095,NaN,Personal & Car related issues,NaN,NaN,NaN,...,NaN,Clear,False,1,48,False,True,False,False,False
4,3.0,102.0,126.0,6.2,29.097896,NaN,More than permitted people in there,NaN,NaN,NaN,...,NaN,Clear,False,1,49,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,1.0,30.0,113.0,6.9,35.800000,NaN,NaN,NaN,440.0,12.85,...,NaN,Partially cloudy,False,22,58,False,False,True,False,False
149996,6.0,141.0,112.0,7.9,33.600000,NaN,NaN,NaN,279.0,39.36,...,NaN,Partially cloudy,False,23,3,False,False,True,False,False
149997,2.0,36.0,5.0,9.9,39.900000,NaN,NaN,NaN,1855.0,38.91,...,NaN,Partially cloudy,False,23,17,False,False,True,False,False
149998,5.0,10.0,169.0,2.6,26.700000,NaN,NaN,NaN,520.0,36.37,...,NaN,Partially cloudy,False,23,21,False,False,True,False,False


In [261]:
df.to_csv('./data/rides_with_weather.csv', index=False)

In [262]:
df.columns

Index(['Vehicle Type', 'Pickup Location', 'Drop Location', 'Avg VTAT',
       'Avg CTAT', 'Reason for cancelling by Customer',
       'Driver Cancellation Reason', 'Incomplete Rides Reason',
       'Booking Value', 'Ride Distance', 'Driver Ratings', 'Customer Rating',
       'temp', 'feelslike', 'dew', 'humidity', 'windgust', 'windspeed',
       'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'conditions',
       'raining', 'hour', 'minute', 'Booking Status_Cancelled by Customer',
       'Booking Status_Cancelled by Driver', 'Booking Status_Completed',
       'Booking Status_Incomplete', 'Booking Status_No Driver Found'],
      dtype='object')

In [263]:
from sklearn.preprocessing import MinMaxScaler

scale_col = [
    'Avg VTAT','Avg CTAT','Booking Value','Ride Distance',
    'Driver Ratings','Customer Rating',
    'temp','feelslike','dew','humidity','windgust','windspeed',
    'sealevelpressure','cloudcover','visibility',
    'solarradiation','solarenergy','uvindex','severerisk'
]

df['hour_sin'] = np.sin(2*np.pi*df['hour']/24.0)
df['hour_cos'] = np.cos(2*np.pi*df['hour']/24.0)
df['minute_sin'] = np.sin(2*np.pi*df['minute']/60.0)
df['minute_cos'] = np.cos(2*np.pi*df['minute']/60.0)
df = df.drop(columns=['hour','minute'])

rad = np.deg2rad(df['winddir'] % 360)
df['winddir_sin'] = np.sin(rad)
df['winddir_cos'] = np.cos(rad)
df = df.drop(columns=['winddir'])

scaler = MinMaxScaler()
df[scale_col] = scaler.fit_transform(df[scale_col])

In [264]:
df[scale_col] = scaler.fit_transform(df[scale_col])
df['Booking Value'] = df['Booking Value'].fillna(0)

In [265]:
df['is_cancelled'] = df[['Booking Status_Cancelled by Customer', 'Booking Status_Cancelled by Driver']].any(axis=1)

In [266]:
df

,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,Reason for cancelling by Customer,Driver Cancellation Reason,Incomplete Rides Reason,Booking Value,Ride Distance,...,Booking Status_Completed,Booking Status_Incomplete,Booking Status_No Driver Found,hour_sin,hour_cos,minute_sin,minute_cos,winddir_sin,winddir_cos,is_cancelled
0,1.0,165.0,3.0,0.488889,0.825714,NaN,NaN,NaN,0.011592,0.754694,...,True,False,False,0.000000,1.000000,0.913545,-0.406737,0.642788,0.766044,False
1,2.0,17.0,88.0,0.361111,0.145714,NaN,NaN,NaN,0.015141,0.781429,...,True,False,False,0.258819,0.965926,-0.500000,-0.866025,0.000000,1.000000,False
2,3.0,163.0,42.0,0.300000,0.551031,NaN,More than permitted people in there,NaN,0.000000,NaN,...,False,False,False,0.258819,0.965926,-0.669131,-0.743145,0.000000,1.000000,True
3,0.0,116.0,81.0,0.200000,0.543317,NaN,Personal & Car related issues,NaN,0.000000,NaN,...,False,False,False,0.258819,0.965926,-0.951057,0.309017,0.000000,1.000000,True
4,3.0,102.0,126.0,0.233333,0.545654,NaN,More than permitted people in there,NaN,0.000000,NaN,...,False,False,False,0.258819,0.965926,-0.913545,0.406737,0.000000,1.000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,1.0,30.0,113.0,0.272222,0.737143,NaN,NaN,NaN,0.092264,0.241837,...,True,False,False,-0.500000,0.866025,-0.207912,0.978148,0.000000,1.000000,False
149996,6.0,141.0,112.0,0.327778,0.674286,NaN,NaN,NaN,0.054176,0.782857,...,True,False,False,-0.258819,0.965926,0.309017,0.951057,0.000000,1.000000,False
149997,2.0,36.0,5.0,0.438889,0.854286,NaN,NaN,NaN,0.427017,0.773673,...,True,False,False,-0.258819,0.965926,0.978148,-0.207912,0.000000,1.000000,False
149998,5.0,10.0,169.0,0.033333,0.477143,NaN,NaN,NaN,0.111190,0.721837,...,True,False,False,-0.258819,0.965926,0.809017,-0.587785,0.000000,1.000000,False


In [267]:
df.to_csv('./data/rides_with_weather_processed.csv', index=False)